# DNS-SEC Example

The following is an example of how DNS-SEC signs keys within the chain of trust. In this simple example:

1) We are using smaller prime numbers to speed up the calculations; normally, the prime numbers would be in the order of 100+ digits long. We are also using abbreviated zone data for the purpose of this exercise for the ikea.se domain. For full details fo the records, you can find zone data here: dnsviz.net/d/ikea.se/dnssec

2) We are using the same key values for zone signing keys (ZSK) and key siging keys (KSK); althought this is permitted under the DNSSEC protocol, it is generally best practice to use separate key values for the KSK and ZSK for security purposes.

3) We will be using RSA/SHA-256 for this exercise, which is the current DNSSEC standard

4) You will need to run SageMath Kernel to verify these calculations


## Zone Data

In [2]:
# Root Server data
Pr = 2**107 - 1
Qr = 2**607 - 1
Nr = Pr*Qr                    # Root modulus
phi_Nr = (Pr - 1)*(Qr - 1)
Er = 101                      # Root encryption exponent
Dr = inverse_mod(Er, phi_Nr)  # Root decryption exponent
root_data = "a.root-servers.net, b.root-servers.net, c.root-servers.net"

# ccTLD data, i.e. Sweden
Pcc = 2**521 - 1
Qcc = 2**127 - 1 
Ncc = Pcc*Qcc                    # ccTLD modulus
phi_Ncc = (Pcc - 1)*(Qcc - 1)
Ecc = 173                      # ccTLD encryption exponent
Dcc = inverse_mod(Ecc, phi_Ncc)  # ccTLD decryption exponent
ccTLD_data = "a.ns.se, b.ns.se, c.ns.se"

# Domain data
# domain data
# Base Values Bob
Pd = 49316606534387222628407057297693523964136820850511818985519515647415558322543
Qd = 91872478880438861140001013686985361958647305204999997818848240040038381660563
phi_Nd = (Pd - 1)*(Qd - 1)
Nd = Pd*Qd                    # domain modulus
Ed = 65537                      # domain encryption exponent
Dd = inverse_mod(Ed, phi_Nd)  # domain decryption exponent
domain_data = "udns1.cscdns.net, udns2.cscdns.uk"

## Zone Data Signing

In [21]:
import hashlib 

def hash_data(zone_data):
    hash = hashlib.sha256()               
    hash.update(zone_data.encode('UTF-8'))  
    return hash.hexdigest()               

ccTLD_hash = hash_data(ccTLD_data)
print("Hash of ccTLD zone data: " + ccTLD_hash + "\n")
domain_hash = hash_data(domain_data)
print("Hash of Ikea's domain data: " + domain_hash + "\n")

#Convert these hashes into integer values
ccTLD_hash_int = int(ccTLD_hash, 16)
domain_hash_int = int(domain_hash, 16)

#Here we will sign our hashes of the zone data with the private key (the Zone Signing Key)
ccTLD_signature = power_mod(ccTLD_hash_int, Dcc, Ncc)
domain_signature = power_mod(domain_hash_int, Dd, Nd) 

print("ccTLD Signature: " + str(ccTLD_signature) + "\n")
print("domain Signature: " + str(domain_signature) + "\n")

Hash of ccTLD zone data: 8bb943b39c114a350ee715de34ac8f8ac7c8a4bf6b16a79c8149a4732b7c28f8

Hash of Ikea's domain data: 512395a1d2aca6a50b09528654e0e3e614b1348adfd61a0b3c2798a8f9d82f1b

ccTLD Signature: 95147196781500944296946541169584512371880948126252795811112131097661215005368983743024507560492607724510177874768310616447788769433317012288542841926627598516090559623954321288988076213782798282

domain Signature: 4385425156426633017965358503193551188580817367346465643174910460031402387674291373082381809978298526692670228233234501371418525190842806324372620195445246



## Key Signing

In [28]:
# We also need to sign the zone public keys, using the private keys of the parent. This is the most importatn feature of DNS
# As it allows us to know that the public key we have matches what the aprent zone has 
# (and this is not a malicious 3rd party server)

ccTLD_public_hash = hash_data(str(Ecc)) #hash of Sweden's public key
domain_public_hash = hash_data(str(Ed)) #hash of Ikea's public key

#Convert to integer values for signing
ccTLD_public_hash_int = int(ccTLD_public_hash,16)
domain_public_hash_int = int(domain_public_hash,16)

#the ccTLD is signed with the root key signing key
ccTLD_public_signature = power_mod(ccTLD_public_hash_int, Dr, Nr) 

#the ikea domain is signed with the ccTLD key signing key
domain_public_signature = power_mod(domain_public_hash_int, Dcc, Ncc)

print("Sweden's public key hash: " + str(ccTLD_public_hash))
print("Ikea's public key hash: " + str(domain_public_hash))
print()
print("Sweden's public key signature: "+ str(ccTLD_public_signature))
print("Ikea's public key signature: "+ str(domain_public_signature))



Sweden's public key hash: 4a8596a7790b5ca9e067da401c018b3206befbcf95c38121854d1a0158e7678a
Ikea's public key hash: d57244ab10a76be13937db76648f504f437e6e0a7c310b4073d6c0bdce3c4ef5

Sweden's public key signature: 5424824200969010175482094077398196239900900504117138350427738265841562973774254499170497259708280430954478553511871483878390161062396054469520746433656865566619703856472083020644127116554510585377847784342266689731
Ikea's public key signature: 633295589923334362768512315073535932215437783270243783725356612715193779841705161928903299011879733769800281891170160915984881834141364829598528064456978764080772071833463135062060707747723765585


## Signature Verification

In [29]:
# Now we are going to pretend ot be a recursive resolver, and one of our clinets wants to visit Ikea.se.
# By obtaining the data and the signatures, all I need to do is check that they match.
# This check will assure me that the data has not been tampered with.

#Knowing the hash function is SHA-256, we verify it ourselves with the zone data
ccTLD_verify_hash = hash_data(ccTLD_data)
domain_verify_hash = hash_data(domain_data)

#And we verify it with the public key hashes we take ourselves (since we can get the public key easily)
ccTLD_publickey_verify = hash_data(str(Ecc))
domain_publickey_verify = hash_data(str(Ed))

# We verify the ZSK signature by decrypting the received signature with the public key of DNS server...
decrypted_ccTLD = hex(power_mod(ccTLD_signature, Ecc, Ncc))[2:]
decrypted_domain = hex(power_mod(domain_signature, Ed, Nd))[2:]

#...and we verify the KSK signature by decrypting the signature with the parent zone's public key
decrypted_public_ccTLD = hex(power_mod(ccTLD_public_signature, Er, Nr))[2:]
decrypted_public_domain = hex(power_mod(domain_public_signature, Ecc, Ncc))[2:]

#Verify that the hash values match:
print("Do the values of the zone signature hashes match?")
print("Is the provided ccTLD zone signature's hash the same as ours? " + str(decrypted_ccTLD==ccTLD_verify_hash))
print("Is the provided Ikea domain signature's hash the same as ours? " + str(decrypted_domain==domain_verify_hash))
print()
print("Do the values of the key signature hashes match?")
print("Is the provided ccTLD key signature's hash the same as ours? " + str(decrypted_public_ccTLD==ccTLD_publickey_verify))
print("Is the provided Ikea key signature's hash the same as ours? " + str(decrypted_public_domain==domain_publickey_verify))


#As these values match, we now know the values received are validated by the parent zone servers.


Do the values of the zone signature hashes match?
Is the provided ccTLD zone signature's hash the same as ours? True
Is the provided Ikea domain signature's hash the same as ours? True

Do the values of the key signature hashes match?
Is the provided ccTLD key signature's hash the same as ours? True
Is the provided Ikea key signature's hash the same as ours? True


## References

The values of E, D, and N (composite of p and q), and the formula for converting between hashes and signatures, were taken from CY202 Cryptography Lab 1 -- part 2 as part of the UC Berkeley MICS curriculum. In this way we did not need to calculate independent values of E & D that were 1 mod N for the purpose of this demonstration, which is for DNSSEC.